# Tutorial: Equivariant Layers from LieGroups

In [23]:
import os
import sys
import warnings

sys.path.append(os.path.dirname(os.getcwd()))
warnings.filterwarnings('ignore')

import geomstats.backend as gs

In [24]:
from emlp.tests

SyntaxError: invalid syntax (990132224.py, line 1)

In [14]:
import emlp
from emlp.reps import T
from emlp.groups import Lorentz
import numpy as np

# repin = 4*T(1)+2*T(0) # 4 four vectors and 2 scalars for the charges
# repout = 3*T(0) # 3 output logits for the 3 classes of collisions
# group = Lorentz()
# model = emlp.nn.EMLP(repin,repout,group=group,num_layers=3,ch=384)

# x = np.random.randn(32,repin(group).size()) # Create a minibatch of data
# y = model(x) # Outputs the 3 class logits

In [4]:

from geomstats.geometry.skew_symmetric_matrices import SkewSymmetricMatrices as SSM

In [8]:
ssm = SSM(n=5)

In [9]:
ssm.basis.shape

(10, 5, 5)

In [10]:
from geomstats.geometry.special_orthogonal import SpecialOrthogonal
from emlp.groups import Group

class GeomstatsGroup(Group):
    def __init__(self, geomstats_group):
        self.d = geomstats_group.dim
        self.lie_algebra = geomstats_group.lie_algebra.basis
        # continuous generators
        super().__init__()
    

INFO: Unable to initialize backend 'tpu_driver': Not found: Unable to find driver in registry given worker: 
INFO: Unable to initialize backend 'gpu': Not found: Could not find registered platform with name: "cuda". Available platform names are: Host Interpreter
INFO: Unable to initialize backend 'tpu': Invalid argument: TpuPlatform is not available.


In [11]:
gs_group = SpecialOrthogonal(n=3)
test_g = GeomstatsGroup(gs_group)

In [16]:
from jax import vmap
import jax.numpy as jnp

In [19]:

def scale_adjusted_rel_error(t1,t2,g):
    error = jnp.sqrt(jnp.mean(jnp.abs(t1-t2)**2))
    tscale = jnp.sqrt(jnp.mean(jnp.abs(t1)**2)) + jnp.sqrt(jnp.mean(jnp.abs(t2)**2))
    gscale = jnp.sqrt(jnp.mean(jnp.abs(g-jnp.eye(g.shape[-1]))**2))
    scale = jnp.maximum(tscale,gscale)
    return error/jnp.maximum(scale,1e-7)

In [20]:
from emlp.reps import *
from emlp.groups import *
from emlp.nn import uniform_rep

def test_sum(G):
    N=5
    rep = T(0,2)+3*(T(0,0)+T(1,0))+T(0,0)+T(1,1)+2*T(1,0)+T(0,2)+T(0,1)+3*T(0,2)+T(2,0)
    rep = rep(G)
    if G.num_constraints()*rep.size()>1e11 or rep.size()**2>10**7: return
    P = rep.equivariant_projector()
    v = np.random.rand(rep.size())
    v = P@v
    gs = G.samples(N)
    gv = (vmap(rep.rho_dense)(gs)*v).sum(-1)
    err = vmap(scale_adjusted_rel_error)(gv,v+jnp.zeros_like(gv),gs).mean()
    assert err<1e-4,f"Symmetric vector fails err {err:.3e} with G={G}"
    


In [21]:
test_sum(test_g)

In [25]:
from emlp.tests.equivariance_tests import test_sum

ModuleNotFoundError: No module named 'emlp.tests'